In [21]:
import os
import requests
from openpyxl import load_workbook

MSP_Data_XLS = r'G:\OneDrive - Esri\LAW\Projects\gis4ocean\nccos_msp\MSP-Workshop-Data_NorthCarolina-SouthCarolina-Virginia.xlsx'
# Create directory if it doesn't exist
output_dir = r'F:\MSP\NC_SC_VA'
os.makedirs(output_dir, exist_ok=True)

errors = []

# Load the workbook
workbook = load_workbook(MSP_Data_XLS)

# Iterate through each sheet in the workbook
for sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]
    
    # Create a subfolder for each sheet
    sheet_dir = os.path.join(output_dir, sheet_name)
    os.makedirs(sheet_dir, exist_ok=True)
    
    # Convert the sheet to a DataFrame
    df = pd.DataFrame(sheet.values)
    df.columns = df.iloc[0]
    df = df[1:]
    
    # Iterate through the DataFrame and download files
    for index, row in df.iterrows():
        url = row['Data Source Link']
        if pd.notna(url):
            try:
                response = requests.get(url)
                response.raise_for_status()
                file_name = os.path.join(sheet_dir, f"file_{index}.zip")
                with open(file_name, 'wb') as file:
                    file.write(response.content)
            except Exception as e:
                errors.append((index, url, str(e)))

# Print errors if any
if errors:
    print("Errors occurred during download:")
    for error in errors:
        print(error)

Errors occurred during download:
(18, 'http://cetsound.noaa.gov/Assets/cetsound/data/CetMap_BIA_WGS84.zip', 'HTTPConnectionPool(host=\'cetsound.noaa.gov\', port=80): Max retries exceeded with url: /Assets/cetsound/data/CetMap_BIA_WGS84.zip (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001BC99D08FD0>: Failed to resolve \'cetsound.noaa.gov\' ([Errno 11001] getaddrinfo failed)"))')
(51, 'https://marinecadastre.gov/downloads/data/mc/AudubonImportantBirdArea.zip', '404 Client Error: The specified blob does not exist. for url: https://marinecadastre.gov/downloads/data/mc/AudubonImportantBirdArea.zip')
(62, 'https://marinecadastre.gov/downloads/data/mc/DeepSeaCoralObservation.zip', '404 Client Error: The specified blob does not exist. for url: https://marinecadastre.gov/downloads/data/mc/DeepSeaCoralObservation.zip')
(63, 'https://marinecadastre.gov/downloads/data/mc/DeepSeaCoralHabitatSuitability.zip', '404 Client Error: The specified blob does not exist. 

In [23]:
error_log_path = os.path.join(output_dir, 'error_log.txt')

with open(error_log_path, 'w') as error_log:
    if errors:
        error_log.write("Errors occurred during download:\n")
        for error in errors:
            error_log.write(f"{error}\n")
    else:
        error_log.write("No errors occurred during download.\n")

print(f"Error log saved to {error_log_path}")

Error log saved to F:\MSP\NC_SC_VA\error_log.txt


: 